# Data Loading and Quality Control

This notebook performs initial data loading and exploratory data analysis (EDA) on the American Gut Project (AGP) dataset.

## Objectives:
- Load feature table, metadata, and taxonomy data
- Inspect data structure and dimensions
- Perform quality control checks
- Generate summary statistics
- Visualize key distributions

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2. Load Data Files

In [5]:
# Define data paths
data_dir = Path('../data/raw')
feature_table_path = data_dir / 'feature-table.tsv'
metadata_path = data_dir / 'metadata.tsv'
taxonomy_path = data_dir / 'taxonomy.tsv'

# Load data
print("Loading feature table...")
feature_table = pd.read_csv(feature_table_path, sep='\t', index_col=0)

print("Loading metadata...")
metadata = pd.read_csv(metadata_path, sep='\t', index_col=0)

print("Loading taxonomy...")
taxonomy = pd.read_csv(taxonomy_path, sep='\t', index_col=0)

print("\n✓ All data loaded successfully!")

Loading feature table...


EmptyDataError: No columns to parse from file

In [ ]:
# OPTIONAL: Filter to subset if dataset is too large
APPLY_FILTER = False  # Set to True to filter
MAX_SAMPLES = 5000

if APPLY_FILTER:
    print(f"Original metadata size: {metadata.shape[0]} samples")
    
    # Filter 1: Keep only fecal/stool samples
    if 'sample_type' in metadata.columns:
        metadata = metadata[metadata['sample_type'].isin(['Stool', 'stool', 'fecal', 'Fecal'])]
        print(f"After sample type filter: {metadata.shape[0]} samples")
    
    # Filter 2: Keep samples with dietary information
    diet_cols = [col for col in metadata.columns if 'diet' in col.lower() or 'coffee' in col.lower()]
    if diet_cols:
        metadata = metadata.dropna(subset=diet_cols, how='all')
        print(f"After diet info filter: {metadata.shape[0]} samples")
    
    # Filter 3: Geographic filter (optional - e.g., USA only)
    # if 'country' in metadata.columns:
    #     metadata = metadata[metadata['country'] == 'USA']
    #     print(f"After country filter: {metadata.shape[0]} samples")
    
    # Filter 4: Limit to max sample size
    if len(metadata) > MAX_SAMPLES:
        metadata = metadata.sample(n=MAX_SAMPLES, random_state=42)
        print(f"Random sample to {MAX_SAMPLES} samples")
    
    # Filter feature table to match metadata
    common_samples = list(set(metadata.index) & set(feature_table.columns))
    feature_table = feature_table[common_samples]
    metadata = metadata.loc[common_samples]
    
    print(f"\n✓ Filtered to {len(metadata)} samples")
    
    # Save filtered data
    filtered_dir = Path('../data/processed')
    filtered_dir.mkdir(parents=True, exist_ok=True)
    metadata.to_csv(filtered_dir / 'metadata_filtered.tsv', sep='\t')
    feature_table.to_csv(filtered_dir / 'feature_table_filtered.tsv', sep='\t')
    print(f"Filtered files saved to {filtered_dir}")
else:
    print("Skipping filter - using full dataset")

## 2.5 Filter to Subset (Optional)

If working with the full AGP dataset, filter to a manageable subset based on:
- Sample type (fecal samples only)
- Metadata completeness
- Geographic location
- Sample size limit

## 3. Initial Data Inspection

### 3.1 Feature Table (OTU/ASV abundance data)

In [ ]:
print("Feature Table Shape:", feature_table.shape)
print(f"Number of samples: {feature_table.shape[1]}")
print(f"Number of features: {feature_table.shape[0]}")
print("\nFirst few rows and columns:")
feature_table.iloc[:5, :5]

### 3.2 Metadata

In [ ]:
print("Metadata Shape:", metadata.shape)
print(f"Number of samples: {metadata.shape[0]}")
print(f"Number of variables: {metadata.shape[1]}")
print("\nColumn names:")
print(metadata.columns.tolist())
print("\nFirst few rows:")
metadata.head()

### 3.3 Taxonomy

In [ ]:
print("Taxonomy Shape:", taxonomy.shape)
print(f"Number of features: {taxonomy.shape[0]}")
print("\nColumn names:")
print(taxonomy.columns.tolist())
print("\nFirst few rows:")
taxonomy.head()

## 4. Data Quality Checks

### 4.1 Missing Values

In [ ]:
# Check missing values in metadata
missing_counts = metadata.isnull().sum()
missing_pct = (missing_counts / len(metadata)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_pct
}).sort_values('Missing Count', ascending=False)

print("Missing values in metadata:")
print(missing_df[missing_df['Missing Count'] > 0])

### 4.2 Sample Overlap Check

In [ ]:
# Check sample overlap between feature table and metadata
feature_samples = set(feature_table.columns)
metadata_samples = set(metadata.index)

overlap = feature_samples & metadata_samples
only_feature = feature_samples - metadata_samples
only_metadata = metadata_samples - feature_samples

print(f"Samples in feature table: {len(feature_samples)}")
print(f"Samples in metadata: {len(metadata_samples)}")
print(f"Samples in both: {len(overlap)}")
print(f"Only in feature table: {len(only_feature)}")
print(f"Only in metadata: {len(only_metadata)}")

### 4.3 Feature Table Summary Statistics

In [ ]:
# Calculate read depth per sample
read_depth = feature_table.sum(axis=0)

print("Read Depth Statistics:")
print(read_depth.describe())

# Calculate features per sample (non-zero features)
features_per_sample = (feature_table > 0).sum(axis=0)

print("\n\nFeatures per Sample Statistics:")
print(features_per_sample.describe())

## 5. Exploratory Visualizations

### 5.1 Read Depth Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(read_depth, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Read Depth')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Read Depth per Sample')

# Box plot
axes[1].boxplot(read_depth, vert=True)
axes[1].set_ylabel('Read Depth')
axes[1].set_title('Read Depth Box Plot')

plt.tight_layout()
plt.show()

### 5.2 Feature Prevalence

In [ ]:
# Calculate feature prevalence (% of samples each feature appears in)
feature_prevalence = ((feature_table > 0).sum(axis=1) / feature_table.shape[1]) * 100

plt.figure(figsize=(10, 6))
plt.hist(feature_prevalence, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Prevalence (%)')
plt.ylabel('Number of Features')
plt.title('Feature Prevalence Distribution')
plt.axvline(x=10, color='r', linestyle='--', label='10% threshold')
plt.legend()
plt.show()

print(f"Features present in >10% of samples: {(feature_prevalence > 10).sum()}")
print(f"Features present in <1% of samples: {(feature_prevalence < 1).sum()}")

### 5.3 Metadata Overview

In [ ]:
# Explore key metadata variables
print("Metadata Info:")
print(metadata.info())
print("\n" + "="*50)
print("\nData types:")
print(metadata.dtypes.value_counts())

## 6. Summary and Next Steps

### Key Findings:
- Document key observations from the EDA
- Note any data quality issues
- Identify variables of interest for coffee consumption analysis

### Next Steps:
- Notebook 02: Define coffee consumption groups from metadata
- Notebook 03: Calculate alpha and beta diversity metrics
- Notebook 04: Perform differential abundance testing